<a href="https://colab.research.google.com/github/ChinmaySomani/Data-Mining-Project/blob/master/Main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!git clone https://github.com/ChinmaySomani/Data-Mining-Project

Cloning into 'Data-Mining-Project'...
remote: Enumerating objects: 19, done.
remote: Counting objects: 100% (19/19), done.
remote: Compressing objects: 100% (17/17), done.
remote: Total 2783 (delta 2), reused 19 (delta 2), pack-reused 2764
Receiving objects: 100% (2783/2783), 814.91 MiB | 33.64 MiB/s, done.
Resolving deltas: 100% (3/3), done.
Checking out files: 100% (2764/2764), done.


In [7]:
import numpy as np
def data_into_lists(filename):
    DataList = []
    with open(filename, 'r') as file:
        for data in file:
            data = data.rstrip('\n')
            dataEntry = data.split(' ')
            #print(dataEntry)
            if dataEntry[0][0] != '#' and len(dataEntry) == 4 and float(dataEntry[3]) == 1:
                lists = []
                for x in range(0,3):
                    lists.append(dataEntry[x])
                DataList.append(lists)
    return DataList
#coords = data_into_lists('Git/Data/gaze/natural_movies_gaze/AAF_beach.coord') 
#coord_matrix = np.array(coords)
#coords[0]
def load_file(filename, data_name):
    coords = data_into_lists(filename)
    coord_matrix = np.array(coords)
    #returns a numpy array of [timestamp, x_loc, y_loc] using coord files 
    return data_name, coord_matrix

In [8]:
#%run data_reader.ipynb
import numpy as np
import scipy.spatial.distance as distance
interval_time = 8000
start_threshold = 0.002
end_threshold = 0.001
def closestTime(coordMatrix, timestamp):
    closeTime = 0
    loca = ()
    minim = 15000
    for coord_matrix in coordMatrix:
        if(abs(int(coord_matrix[0]) - timestamp) < minim):
            minim = abs(int(coord_matrix[0]) - timestamp)
            closeTime = int(coord_matrix[0])
            loca = (int(float(coord_matrix[1])), int(float(coord_matrix[2])))
    return closeTime, loca

def intervals(filename):
    name, coords = load_file(filename, 'test')
    interval = []
    start_time = int(coords[0][0])
    #last = coords[-1][0]
    while start_time < int(coords[-1][0])-1:
        final_start_time, start_loc = closestTime(coords, start_time)
        end_time = start_time + interval_time
        final_end_time, end_loc = closestTime(coords, end_time)
        if(final_end_time == final_start_time):
            end_time = final_start_time + 10000
            final_end_time, end_loc = closestTime(coords, end_time)
        interval.append((start_loc, end_loc, int(final_end_time - final_start_time), final_start_time))
        start_time = start_time + (interval_time/2)
    inteval = np.array(interval, dtype=object)
    return interval

def velocities(interval):
    velocity = []
    for inter in interval:
        if(len(inter) and inter[2] and len(inter[0]) and len(inter[1])):
            #print(inter)
            #if(inter[2] != 0):
            dist = pow((pow(inter[0][0] - inter[1][0],2) + pow(inter[0][1] - inter[1][1],2)), 1/2)
            vel = int(dist)/inter[2]
            velocity.append(vel)
    velocity = np.array(velocity)
    return velocity

def extract_saccades(velocity, interval):  #videopath
    saccade = False
    data = []
    start_location = None
    for i in range(0, len(velocity)-1):
        acc = abs(velocity[i+1] - velocity[i])
        if saccade == False:
            if acc >= start_threshold:
                saccade = True
                start_location = interval[i][0]
                timestamp = interval[i][3]
        else:
            if acc <= end_threshold:
                saccade = False
                label = interval[i][1]
                data.append((start_location, label, timestamp)) #videopath
                start_location = None
    #return ((start_loc, label, timeframe))            
    return data

    #interval1 = intervals('Git/Data/gaze/natural_movies_gaze/AAF_beach.coord')
#vel = velocities(interval1)
#data = extract_saccades(vel, interval1)
#print(len(data))
#print(len(vel))
#print(data[0])

In [9]:
!pip install sk-video

     |████████████████████████████████| 2.3MB 10.3MB/s 


In [10]:

import imageio
import numpy as np
from PIL import Image
import skvideo.io
import cv2 
import os

def frameExtractor(filename, timestamp):
    vid = cv2.VideoCapture(filename)
    framePerSec = vid.get(cv2.CAP_PROP_FPS)
    framePerSec = framePerSec/1000
    frameNumber = 0
    if timestamp >= 1 : 
        frameNumber = int(framePerSec * timestamp) - 1
    return frameNumber

def getFrame(timestamp, x, y, videopath):
    width = 1280
    height = 720
    videodata = skvideo.io.vread(videopath)
    frameNum = frameExtractor(videopath, timestamp/1000)
    frame = videodata[frameNum-1]
    imageio.imsave('frame.jpg', frame)

#def createDirectory(directory):
 #   import os
 #   #if directory is not present then make it
 #   if os.path.exists(os.path.dirname(directory)) == False :
 #       os.makedirs(os.path.dirname(directory))

def tripleCrop(imagepath, coord):
    #createDirectory(directory)
    x, y = coord
    cropDim = 32
    for i in range(0,3):
        im = Image.open(imagepath)
        cropDim = cropDim*2 
        leftCor = x - cropDim
        rightCor = x + cropDim
        upperCor = y - cropDim
        lowerCor = y + cropDim
        im = im.crop((leftCor, upperCor, rightCor, lowerCor))
        im = im.resize((64, 64))
        #path = directory + name + '_' + str(i) + '.png'
        im.save('triplecroppedimage'+str(i)+'.jpg')

def combineTripleCrop(entry):
    #createDirectory('TrainingData')
    crop0 = imageio.imread('triplecroppedimage0.jpg')
    crop1 = imageio.imread('triplecroppedimage1.jpg')
    crop2 = imageio.imread('triplecroppedimage2.jpg')
    H = 64
    W = 64*3
    C = 3
    triple = np.zeros((H,W,C), dtype=np.uint8)
    triple[:, :W//3, :] += crop0
    triple[:, W//3 : 2*W//3, :] += crop1
    triple[:, 2*W//3:, :] += crop2
    imageio.imsave('TrainingData/combine'+ str(entry) + '.jpg', triple)

In [13]:
#%run data_reader.ipynb
#%run Sacade_extractor.ipynb
#%run VideoUtilites.ipynb
import csv
import os
import numpy as np

import imageio
import numpy as np
from PIL import Image


def maketraindata(file, video, filecount):
  interval = intervals(file)
  velocity = velocities(interval)
  data = extract_saccades(velocity, interval)

  data = np.array(data)
  data.shape

  w = csv.writer(open("labels.csv", "a"))
  entry = 0
  for point, label, timestamp in data:
    if timestamp and label and point :
      x, y = point
      getFrame(timestamp/1000, x, y, video)
      tripleCrop('frame.jpg', point)
   # combineTripleCrop(entry)


      label_x, label_y = label
      CropDim = 512//2
      if label_y > y + CropDim:
        label_y = y + CropDim
      elif label_y < y - CropDim:
        label_y = y - CropDim
      if label_x > x + CropDim:
        label_x = x + CropDim
      elif label_x < x - CropDim:
        label_x = x - CropDim
    
      label_y_LeftBigCrop = label_y - y + CropDim
      label_x_LeftBigCrop = label_x - x + CropDim
      label_x_LeftBigCrop /= 8
      label_y_LeftBigCrop /= 8  
      fx = round(label_x_LeftBigCrop)
      fy = round(label_y_LeftBigCrop)
      label = (fx, fy)
      w.writerow([entry,'TrainingData/combine' + str(entry)+'.jpg', label[0], label[1]])

      crop0 = imageio.imread('triplecroppedimage0.jpg')
      crop1 = imageio.imread('triplecroppedimage1.jpg')
      crop2 = imageio.imread('triplecroppedimage2.jpg')
      H = 64
      W = 64*3
      C = 3
      triple = np.zeros((H,W,C), dtype=np.uint8)
      triple[:, :W//3, :] += crop0
      triple[:, W//3 : 2*W//3, :] += crop1
      triple[:, 2*W//3:, :] += crop2

      if fx <= 16 :
        if fy <= 16 :
          imageio.imsave('TrainingData/0/combine'+ str(filecount) + 'o' + str(entry) + '.jpg', triple)
        if fy > 16 and fy <= 32 :
          imageio.imsave('TrainingData/1/combine'+ str(filecount) + 'o' + str(entry) + '.jpg', triple)
        if fy > 32 and fy <= 48 :
          imageio.imsave('TrainingData/2/combine'+ str(filecount) + 'o' + str(entry) + '.jpg', triple)
        if fy > 48 and fy <= 64 :
          imageio.imsave('TrainingData/3/combine'+ str(filecount) + 'o' + str(entry) + '.jpg', triple)
      if fx > 16 and fx <=32 :
        if fy <= 16 :
          imageio.imsave('TrainingData/4/combine'+ str(filecount) + 'o' + str(entry) + '.jpg', triple)
        if fy > 16 and fy <= 32 :
          imageio.imsave('TrainingData/5/combine'+ str(filecount) + 'o' + str(entry) + '.jpg', triple)
        if fy > 32 and fy <= 48 :
          imageio.imsave('TrainingData/6/combine'+ str(filecount) + 'o' + str(entry) + '.jpg', triple)
        if fy > 48 and fy <= 64 :
          imageio.imsave('TrainingData/7/combine'+ str(filecount) + 'o' + str(entry) + '.jpg', triple)  
      if fx >32 and fx <=48 :
        if fy <= 16 :
          imageio.imsave('TrainingData/8/combine'+  str(filecount) + 'o' +str(entry) + '.jpg', triple)
        if fy > 16 and fy <= 32 :
          imageio.imsave('TrainingData/9/combine'+ str(filecount) + 'o' + str(entry) + '.jpg', triple)
        if fy > 32 and fy <= 48 :
          imageio.imsave('TrainingData/10/combine'+  str(filecount) + 'o' +str(entry) + '.jpg', triple)
        if fy > 48 and fy <= 64 :
          imageio.imsave('TrainingData/11/combine'+ str(filecount) + 'o' + str(entry) + '.jpg', triple)
      if fx > 48 and fx <= 64 :
        if fy <= 16 :
          imageio.imsave('TrainingData/12/combine'+ str(filecount) + 'o' + str(entry) + '.jpg', triple)
        if fy > 16 and fy <= 32 :
          imageio.imsave('TrainingData/13/combine'+ str(filecount) + 'o' + str(entry) + '.jpg', triple)
        if fy > 32 and fy <= 48 :
          imageio.imsave('TrainingData/14/combine'+ str(filecount) + 'o' + str(entry) + '.jpg', triple)
        if fy > 48 and fy <= 64 :
          imageio.imsave('TrainingData/15/combine'+ str(filecount) + 'o' + str(entry) + '.jpg', triple)
    
      print(entry)
      entry = entry + 1

In [17]:
filecount = 0
for filename in os.listdir('./Data-Mining-Project/Data/gaze/natural_movies_gaze'):
    video_start_index = filename.find('_')+1
    video_end_index = filename.find('.coord')
    video_path = './Data-Mining-Project/Data/movies_mpg/' + filename[video_start_index: video_end_index] + '.mpg'
    maketraindata('./Data-Mining-Project/Data/gaze/natural_movies_gaze/'+filename, video_path, filecount)
    filecount = filecount + 1
    print(filecount)

75
2


KeyboardInterrupt: ignored

In [23]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

trainingdatagen = ImageDataGenerator(validation_split=0.1)
traingenerator = trainingdatagen.flow_from_directory('./TrainingData', target_size = (64,192), classes=['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15'], batch_size = 20, class_mode = 'categorical')

Found 114 images belonging to 16 classes.


In [24]:
import tensorflow as tf

model = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(16, (3,3), padding='same', activation='relu', input_shape=(64, 192, 3)),
        tf.keras.layers.MaxPooling2D((2,2),2),
        tf.keras.layers.Conv2D(16, (3,3), padding='same', activation='relu'),
        tf.keras.layers.MaxPooling2D((2,2),2),
        tf.keras.layers.Conv2D(16, (3,3), padding='same', activation='relu'),
        tf.keras.layers.MaxPooling2D((2,2),2),
        tf.keras.layers.Conv2DTranspose(16, (4,4), padding='same',strides=2, activation='relu'),
        tf.keras.layers.Conv2DTranspose(1, (6,6), padding='same',strides=4, activation='sigmoid'),
        tf.keras.layers.Conv2D(8, (3,3), padding='same',strides=1, activation='relu'),
        tf.keras.layers.MaxPooling2D((2,2),2),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(16, activation='softmax')
])

In [25]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 64, 192, 16)       448       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 32, 96, 16)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 32, 96, 16)        2320      
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 16, 48, 16)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 16, 48, 16)        2320      
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 8, 24, 16)         0         
_________________________________________________________________
conv2d_transpose_2 (Conv2DTr (None, 16, 48, 16)       

In [26]:
from tensorflow.keras.optimizers import RMSprop
model.compile(optimizer=RMSprop(lr=0.001),loss='categorical_crossentropy', metrics = ['accuracy'])

In [27]:
#from tensorflow.keras.optimizers import RMSprop
history = model.fit(
      traingenerator,
      steps_per_epoch=8,  
      epochs=15,
      verbose=1)

Epoch 1/15
6/8 [=====================>........] - 2s 266ms/step - loss: 16.9954 - accuracy: 0.1930


In [28]:
import tensorflow as tf

from tensorflow.keras import layers
from tensorflow.keras import Model
pretrain_model = tf.keras.applications.VGG16(
    include_top=False, weights='imagenet', input_shape=(64, 192, 3), classifier_activation='softmax'
)
for layer in pretrain_model.layers:
  layer.trainable = False

last_layer = pretrain_model.get_layer('block5_pool')
print('last layer output shape: ', last_layer.output_shape)
last_output = last_layer.output

from tensorflow.keras.optimizers import RMSprop

# Flatten the output layer to 1 dimension
x = layers.Flatten()(last_output)
# Add a fully connected layer with 1,024 hidden units and ReLU activation
x = layers.Dense(1024, activation='relu')(x)                 
# Add a final sigmoid layer for classification
x = layers.Dense  (16, activation='softmax')(x)           

vmodel = Model( pretrain_model.input, x) 

vmodel.compile(optimizer = RMSprop(lr=0.0001), 
              loss = 'categorical_crossentropy', 
              metrics = ['accuracy'])
vmodel.summary()

last layer output shape:  (None, 2, 6, 512)
Model: "functional_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 64, 192, 3)]      0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 64, 192, 64)       1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 64, 192, 64)       36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 32, 96, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 32, 96, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 32, 96, 128)       147584    
_________________________________________________________________
block2_poo

In [30]:
vhistory = vmodel.fit(
      traingenerator,
      steps_per_epoch=8,  
      epochs=15,
      verbose=1)

Epoch 1/15
6/8 [=====================>........] - 11s 2s/step - loss: 19.6328 - accuracy: 0.1491
